# Xopt Evaluator Basic Usage 

The `Evaluator` handles the execution of the user-provided `function` with optional `function_kwags`, asyncrhonously and parallel, with exception handling. 

In [ ]:
from xopt import Xopt, Evaluator, VOCS
from xopt.generators.random import RandomGenerator

# Usage with a parallel executor.
from xopt import AsynchronousXopt

import pandas as pd

from time import sleep
from numpy.random import randint

from typing import Dict

import numpy as np

from concurrent.futures import ProcessPoolExecutor

# needed for macos
import platform

if platform.system() == "Darwin":
    import multiprocessing

    multiprocessing.set_start_method("fork")

In [ ]:
np.random.seed(666)  # for reproducibility

Define a custom function `f(inputs: Dict) -> outputs: Dict`. 

In [ ]:
def f(inputs: Dict, enable_errors=True) -> Dict:
    sleep(randint(1, 5) * 0.1)  # simulate computation time
    # Make some occasional errors
    if enable_errors and np.any(inputs["x"] > 0.8):
        raise ValueError("x > 0.8")

    return {"f1": inputs["x"] ** 2 + inputs["y"] ** 2}

Define variables, objectives, constraints, and other settings (VOCS)

In [ ]:
vocs = VOCS(variables={"x": [0, 1], "y": [0, 1]}, objectives={"f1": "MINIMIZE"})
vocs

This can be used to make some random inputs for evaluating the function. 

In [ ]:
in1 = vocs.random_inputs()[0]

f(in1, enable_errors=False)

In [ ]:
# Add in occasional errors.
try:
    f({"x": 1, "y": 0})
except Exception as ex:
    print(f"Caught error in f: {ex}")

In [ ]:
# Create Evaluator
ev = Evaluator(function=f)

In [ ]:
# Single input evaluation
ev.evaluate(in1)

In [ ]:
# Dataframe evaluation
in10 = pd.DataFrame({"x": np.linspace(0, 1, 10), "y": np.linspace(0, 1, 10)})
ev.evaluate_data(in10)

In [ ]:
# Dataframe evaluation, vectorized
ev.vectorized = True
ev.evaluate_data(in10)

# Executors

In [ ]:
MAX_WORKERS = 10

In [ ]:
# Create Executor instance
executor = ProcessPoolExecutor(max_workers=MAX_WORKERS)
executor

In [ ]:
# Dask (Optional)
# from dask.distributed import Client
# import logging
# client = Client( silence_logs=logging.ERROR)
# executor = client.get_executor()
# client

In [ ]:
# This calls `executor.map`
ev = Evaluator(function=f, executor=executor, max_workers=MAX_WORKERS)

In [ ]:
# This will run in parallel
ev.evaluate_data(in10)

# Evaluator in the Xopt object

In [ ]:
X = Xopt(
    generator=RandomGenerator(vocs=vocs), evaluator=Evaluator(function=f), vocs=vocs
)
X.strict = False

# Evaluate to the evaluator some new inputs
X.evaluate_data(X.vocs.random_inputs(4))

## Asynchronous Xopt
Instead of waiting for evaluations to be finished, AsynchronousXopt can be used to generate candidates while waiting for other evaluations to finish (requires parallel execution). In this case, calling ```X.step()``` generates and executes a number of candidates that are executed in parallel using python ```concurrent.futures``` formalism. Calling ```X.step()``` again will generate and evaluate new points based on finished futures asynchronously.

In [ ]:
executor = ProcessPoolExecutor(max_workers=MAX_WORKERS)

X2 = AsynchronousXopt(
    generator=RandomGenerator(vocs=vocs),
    evaluator=Evaluator(function=f, executor=executor, max_workers=MAX_WORKERS),
    vocs=vocs,
)
X2.strict = False

In [ ]:
X2.step()

In [ ]:
for _ in range(20):
    X2.step()

len(X2.data)

In [ ]:
X2.data.plot.scatter("x", "y")

In [ ]:
# Asynchronous, Vectorized
X2 = AsynchronousXopt(
    generator=RandomGenerator(vocs=vocs),
    evaluator=Evaluator(function=f, executor=executor, max_workers=MAX_WORKERS),
    vocs=vocs,
)
X2.evaluator.vectorized = True
X2.strict = False

# This takes fewer steps to achieve a similar number of evaluations
for _ in range(3):
    X2.step()

len(X2.data)